<a href="https://colab.research.google.com/github/PilipenkoTatiana/Mice-USVs-and-behaviors/blob/main/Sounds_behaviors_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%% import packages
import pandas as pd
import os
import glob
import nltk
import matplotlib.pyplot as plt

In [ ]:
#%% select study group and sequence
study_group = 'VH'  #CBD - drugs, VH - no drugs
sequence_length = 3 #1 #2 #3 #4

In [ ]:
#%% read excel with behavior-sound


os.chdir('...\\adults_excel_sound_behavior_merged')
excel_file_list = glob.glob('*.xlsx')
selected_sheet_names = [ x for x in excel_file_list if "$" not in x ]
selected_sheet_names = [ x for x in excel_file_list if study_group in x ]
selected_sheet_names = [k for k in selected_sheet_names if 'topo' in k]


# define the behavior list
behavior_list = ["anogenital sniffing", "contact", "exploring", "immobile", "nose-nose sniffing", "self grooming", "sniffing body", "following"]



In [ ]:
counter = 1
#counter_all_mice = pd.DataFrame()
for mouse in selected_sheet_names:

    os.chdir('...\\adults_excel_sound_behavior_merged')

    #df = pd.read_excel(excel_file_name, sheet_name= mouse)
    df = pd.read_excel(mouse)

    # drop rows with nan in behavior-sound (sometimes we have behaviors without sounds)
    df = df.dropna()
    # reset index
    df = df.reset_index(drop=True)

    df['end time'] = df['start time'] + df['Duration_Sound']

    # divide in sentences
    df_time = df[['TIPOLOGIA', 'start time', 'end time', 'Behavior']]
    df_time = df_time[df_time['TIPOLOGIA'].notna()]

    if len(df_time) > 1:
        sentence = 0
        seconds = 0.55
        sentence_label = []
        sentence_label.append(0)
        for m in range(0,len(df_time)-1):
            if df['start time'][m+1] - df['end time'][m] < seconds:
                sentence_label.append(sentence)
            else:
                sentence += 1
                sentence_label.append(sentence)

        df_time['Sentence_Label'] = sentence_label

        #data cleaning
        try:
            df_time = df_time[~df_time['TIPOLOGIA'].str.startswith('?')]
            df_time = df_time[~df_time['TIPOLOGIA'].str.endswith('*')]
            df_time = df_time.reset_index()
            for i in range(0,len(df_time)):
                df_time['TIPOLOGIA'][i] = df_time['TIPOLOGIA'][i].rstrip()
        except:
            df_time = df_time[~df_time['TIPOLOGIE'].str.startswith('?')]
            df_time = df_time[~df_time['TIPOLOGIE'].str.endswith('*')]
            df_time = df_time.reset_index()
            for i in range(0,len(df_time)):
                df_time['TIPOLOGIE'][i] = df_time['TIPOLOGIE'][i].rstrip()


        sentences = {}
        for l in range(0,sentence+1):
            sentences[l] = df_time.loc[df_time['Sentence_Label'] == l]


        # drop sentences smaller than the sequence length
        for u in range(0,len(sentences)):
            if len(sentences[u]) < sequence_length:
                del sentences[u]

        #execute the following code only if there are sentences with at least two sounds
        if len(sentences) > 0:
            # reset keys of the sentences dictionary (deleting sentences can create gaps in keys)
            sentences = {p: sentences[b] for p, b in enumerate(sorted(sentences.keys()))}

            for snt in range(0, len(sentences)):
                for behavior in behavior_list:
                    temporary_df = sentences[snt].loc[sentences[snt]['Behavior'] == behavior]
                    output = nltk.FreqDist(nltk.ngrams(list(temporary_df['TIPOLOGIA']), sequence_length))
                    pattern_sequence = pd.DataFrame(output.items(), columns=['pattern', 'frequency'])


                    try:
                        globals()['behavior_%s'%behavior] = pd.merge(globals()['behavior_%s'%behavior], pattern_sequence, on=['pattern'],how='outer')
                        globals()['behavior_%s' % behavior].rename(columns={'frequency': mouse}, inplace = True)
                        # there is a bug putting "pattern" in a random position, let's put it as first column:
                        first_column =  globals()['behavior_%s' % behavior].pop('pattern')
                        globals()['behavior_%s' % behavior].insert(0, 'pattern', first_column)
                    except:
                        globals()['behavior_%s' % behavior] = pattern_sequence
                        globals()['behavior_%s' % behavior].rename(columns={'frequency': mouse}, inplace = True)
                        # there is a bug putting "pattern" in a random position, let's put it as first column:
                        first_column =  globals()['behavior_%s' % behavior].pop('pattern')
                        globals()['behavior_%s' % behavior].insert(0, 'pattern', first_column)

    counter += 1


In [ ]:
for behavior in behavior_list:
    globals()['behavior_%s' % behavior]['count'] = globals()['behavior_%s' % behavior].sum(axis = 1)
    globals()['behavior_%s' % behavior] = globals()['behavior_%s' % behavior][['pattern', 'count']]


for behavior in behavior_list:
    globals()['behavior_%s' % behavior].to_excel("dataframe_" + behavior + ".xlsx")




In [ ]:
# filter the dataframes (they are too large for a decent representation)
# from now on, behavior is no longer necessary
os.chdir('...\\adults_excel_sound_behavior_merged')
final_df = pd.read_excel('sounds_merged_pivot.xlsx', sheet_name=str(sequence_length) + ' sounds - ' + study_group)
# define percentage rows to display (they are too many)
percentage_patterns_total = 0.05
number_of_patterns_total = int(percentage_patterns_total*len(final_df))
final_df = final_df.head(number_of_patterns_total)
final_df = final_df.sort_values('pattern', ascending = True)
final_df =  final_df.reset_index(drop=True)


In [ ]:
# replace the name of the sounds with their abbreviations (for an easier to read visualization)
final_df['pattern'] = final_df['pattern'].str.replace('chevron','CH')
final_df['pattern'] = final_df['pattern'].str.replace('complex','CX')
final_df['pattern'] = final_df['pattern'].str.replace('composite','CM')
final_df['pattern'] = final_df['pattern'].str.replace('downward','D')
final_df['pattern'] = final_df['pattern'].str.replace('flat','F')
final_df['pattern'] = final_df['pattern'].str.replace('frequency steps','FS')
final_df['pattern'] = final_df['pattern'].str.replace('harmonics','H')
final_df['pattern'] = final_df['pattern'].str.replace('short','S')
final_df['pattern'] = final_df['pattern'].str.replace('two-syllable','TS')
final_df['pattern'] = final_df['pattern'].str.replace('upward','U')



In [ ]:
# final plot
fig = plt.figure() # everytime we start a new picture, we define a new fig
width = .35
final_df['count'].plot(kind='bar', width = width, color = 'green', figsize=(30,25))
final_df['count'].plot(kind='line',secondary_y=False, color='red', linewidth=4)
ax = plt.gca()
ax.set_xticklabels(final_df['pattern'], fontsize=50)
ax.tick_params(axis='y', which='major', labelsize=50)  # fix font size y axis
plt.xticks(rotation = 90)
ax.set_ylabel('Count', fontsize=60)
ax.set_xlabel('Sounds',fontsize=60)
#ax.set_title('Adult ' + study_group + ' mice sounds total for ' + behavior + ' - sequence length: ' + str(sequence_length), fontsize=20)
ax.set_facecolor('xkcd:white')


In [ ]:

#fig.set_size_inches(17.5, 13)
fig.tight_layout()
# create folder for the selected time frame
os.chdir('...\\'+study_group)

try:
    os.mkdir(str(seconds) + ' seconds')
except:
    pass

os.chdir('...\\'+study_group+'\\' + str(seconds) + ' seconds')
'''
if behavior in ['sniffing body', 'anogenital sniffing', 'nose-nose sniffing']:
    plt.ylim(0,350)
elif behavior == 'exploring':
    plt.ylim(0,140)
elif behavior == 'contact':
    plt.ylim(0,50)
'''
fig.tight_layout()
#fig.savefig('adults_total_hist_' + study_group + " " + str(sequence_length) + ' - ' + str(seconds) + ' seconds' +'.pdf', bbox_inches='tight')
plt.show()
plt.clf()

#final_df.to_excel('Final_DF ' + study_group + ' - ' + str(sequence_length) + ' sounds.xlsx')

# eliminate the variables globals()['behavior_%s' % behavior] (necessary to re-run the code without opening a new console)

for behavior in behavior_list:
    del globals()['behavior_%s' % behavior]
